<a href="https://colab.research.google.com/github/EyosiyasE/EyosiyasE/blob/main/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q qiskit qiskit-nature==0.7.2 pyscf matplotlib pandas qiskit-aer
!pip install -q pymatgen


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 40.5 MB/s eta 0:00:00


In [ ]:
# --- Core imports for Qiskit Nature simulations ---

from qiskit_nature.second_q.drivers import PySCFDriver, Molecule
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA, SLSQP, SPSA
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit.primitives import Estimator

def make_system(name):
    if name == "LiFePO4":
        charge, multiplicity = 0, 5
        geom = [
            ("Fe",(0.0,0.0,0.0)),
            ("O",( 2.1,0.0,0.0)), ("O",(-2.1,0.0,0.0)),
            ("O",(0.0, 2.1,0.0)), ("O",(0.0,-2.1,0.0)),
            ("O",(0.0,0.0, 2.1)), ("O",(0.0,0.0,-2.1)),
            ("P",(0.0,0.0,3.4)),
            ("O",( 0.889, 0.889,4.289)),
            ("O",( 0.889,-0.889,2.511)),
            ("O",(-0.889, 0.889,2.511)),
            ("O",(-0.889,-0.889,4.289)),
            ("Li",(0.0,0.0,5.2)),
        ]
    elif name == "NaFePO4":
        charge, multiplicity = 0, 5
        geom = [
            ("Fe",(0.0,0.0,0.0)),
            ("O",( 2.1,0.0,0.0)), ("O",(-2.1,0.0,0.0)),
            ("O",(0.0, 2.1,0.0)), ("O",(0.0,-2.1,0.0)),
            ("O",(0.0,0.0, 2.1)), ("O",(0.0,0.0,-2.1)),
            ("P",(0.0,0.0,3.4)),
            ("O",( 0.889, 0.889,4.289)),
            ("O",( 0.889,-0.889,2.511)),
            ("O",(-0.889, 0.889,2.511)),
            ("O",(-0.889,-0.889,4.289)),
            ("Na",(0.0,0.0,5.2)),
        ]
    elif name == "MgFePO4":
        charge, multiplicity = 0, 6
        geom = [
            ("Fe",(0.0,0.0,0.0)),
            ("O",( 2.1,0.0,0.0)), ("O",(-2.1,0.0,0.0)),
            ("O",(0.0, 2.1,0.0)), ("O",(0.0,-2.1,0.0)),
            ("O",(0.0,0.0, 2.1)), ("O",(0.0,0.0,-2.1)),
            ("P",(0.0,0.0,3.4)),
            ("O",( 0.889, 0.889,4.289)),
            ("O",( 0.889,-0.889,2.511)),
            ("O",(-0.889, 0.889,2.511)),
            ("O",(-0.889,-0.889,4.289)),
            ("Mg",(0.0,0.0,5.2)),
        ]
    else:
        raise ValueError("Unknown name")
    mol = Molecule(geometry=geom, charge=charge, multiplicity=multiplicity)
    return mol
